In [17]:
import pandas as pd
#import matplotlib.pyplot as plt
#import matplotlib.dates as mdates
import numpy as np
#from datetime import datetime
#from datetime import date

In [18]:
read_pars = pd.read_csv('StockReorderlocations_2022-01-21.csv',header=0,dtype=str)
read_pars.rename(columns={'ProductSKU':'SKU','MinimumBeforeReorder':'Par'},inplace=True)
read_pars.drop(['StockLocator','PickZones'],inplace=True,axis=1)
read_pars.Par = read_pars.Par.astype(float)
read_pars.Par = read_pars.Par.astype(int)
read_pars.ReorderQuantity = read_pars.ReorderQuantity.astype(float)
read_pars.ReorderQuantity = read_pars.ReorderQuantity.astype(int)
read_pars = read_pars.drop(columns=['Action','ReorderQuantity'])
read_pars.head()

,SKU,ProductName,Location,Par
0,086785024684,Drumshanbo Irish Gin 375ml,Oak Hill,1
1,086785024684,Drumshanbo Irish Gin 375ml,Pleasant Valley,4
2,086785024684,Drumshanbo Irish Gin 375ml,Airport,1
3,786992000609,Chairmans Spiced Rum 750ml,Warehouse,4
4,786992000609,Chairmans Spiced Rum 750ml,Oak Hill,2


In [19]:
df = pd.read_csv('AvailabilityReport_2022-02-02.csv')
df.head(2)

,Category,SKU,ProductName,Location,Bin,BatchSerialNumber,ExpiryDate,Discount,StockValue,OnHand,...,Allocated,Status,Unit,Family,StockLocator,Brand,PriceTier1,PriceTier2,PriceTier3,NextDeliveryDate
0,Non Alcoholic Beverage,777823111113,1 Water 355ml,Airport,NaN,NaN,NaN,NaN,24.1995,123.0,...,0.0,ACTIVE,Item,NaN,NaN,NaN,1.5,0.95,0.95,NaN
1,Non Alcoholic Beverage,777823111113,1 Water 355ml,Oak Hill,NaN,NaN,NaN,NaN,0.5357,1.0,...,0.0,ACTIVE,Item,NaN,NaN,NaN,1.5,0.95,0.95,NaN


In [20]:
df = df.drop(columns=['Bin','BatchSerialNumber','ExpiryDate','Discount','StockValue','VolumeOnHand','Available','OnOrder','InTransit','Allocated','Status','Unit','Family','StockLocator','Brand','PriceTier1','PriceTier2','PriceTier3','NextDeliveryDate'])
df.head(2)

,Category,SKU,ProductName,Location,OnHand
0,Non Alcoholic Beverage,777823111113,1 Water 355ml,Airport,123.0
1,Non Alcoholic Beverage,777823111113,1 Water 355ml,Oak Hill,1.0


In [21]:
ans = df.merge(right=read_pars,on=['SKU','ProductName','Location'])
ans.head()

,Category,SKU,ProductName,Location,OnHand,Par
0,Bar Tools,1234500000000,12/24 Mixing Tin Sets,Pleasant Valley,0.0,4
1,Tequila,7503013136221,123 Organic Anejo Tequila 750ml,Warehouse,0.0,2
2,Tequila,7503013136207,123 Organic Blanco Tequila 750ml,Airport,2.0,0
3,Tequila,7503013136207,123 Organic Blanco Tequila 750ml,Oak Hill,1.0,0
4,Tequila,7503013136207,123 Organic Blanco Tequila 750ml,Pleasant Valley,3.0,1


In [22]:
host_store = 'Oak Hill'

ans['Shy'] = ans.Par - ans.OnHand
print(ans.head(10))
ans['Send'] = np.nan
for i, row in ans.iterrows():
    temp = ans[(ans.Location == host_store) & (ans.ProductName == row.ProductName)]
    if len(temp) > 0 and row.Shy > 0 and temp.iloc[0].Shy < 0:
        #print(i,'yes')
        if row.Shy <= (temp.iloc[0].Shy*-1):
            #print(i,'if')
            ans.at[i,'Send'] = row.Shy
            #row.iloc[i].['Send']='FUCK'
        else:
            #print(i,'else')
            ans.at[i,'Send'] = temp.iloc[0].Shy*-1
            #row['Send']='SHIT'


#ans['Candidate'] = ans.apply(checkstock,axis=1)

    Category            SKU                         ProductName  \
0  Bar Tools  1234500000000               12/24 Mixing Tin Sets   
1    Tequila  7503013136221     123 Organic Anejo Tequila 750ml   
2    Tequila  7503013136207    123 Organic Blanco Tequila 750ml   
3    Tequila  7503013136207    123 Organic Blanco Tequila 750ml   
4    Tequila  7503013136207    123 Organic Blanco Tequila 750ml   
5    Tequila  7503013136207    123 Organic Blanco Tequila 750ml   
6    Tequila  7503013136214  123 Organic Reposado Tequila 750ml   
7    Tequila  7503013136214  123 Organic Reposado Tequila 750ml   
8    Tequila  7503013136214  123 Organic Reposado Tequila 750ml   
9    Tequila  7503013136214  123 Organic Reposado Tequila 750ml   

          Location  OnHand  Par  Shy  
0  Pleasant Valley     0.0    4  4.0  
1        Warehouse     0.0    2  2.0  
2          Airport     2.0    0 -2.0  
3         Oak Hill     1.0    0 -1.0  
4  Pleasant Valley     3.0    1 -2.0  
5        Warehouse     0.0  

In [23]:
ans[ans.Send.isnull()==False].head(40)
ans.to_csv('InventoryShuffle.csv')

# Again but with *my* pars (shit this doesn't show warehouse)


In [24]:
reader = pd.read_csv('ParAnalysis.csv',dtype=str)
#print(reader.head())
my_pars = reader[['ProductSKU','Product','Customer','75']].copy()
my_pars.rename(columns={'ProductSKU':'SKU','Product':'ProductName','75':'Par','Customer':'Location'},inplace=True)
my_pars.Par = my_pars.Par.astype(float)
my_pars.Par = my_pars.Par.astype(int)
my_pars.head()

,SKU,ProductName,Location,Par
0,7503013136207,123 Organic Blanco Tequila 750ml,Customer Square 1,0
1,7503013136207,123 Organic Blanco Tequila 750ml,Customer Square 2,0
2,7503013136207,123 Organic Blanco Tequila 750ml,Customer Square 3,0
3,7503013136214,123 Organic Reposado Tequila 750ml,Customer Square 1,0
4,7503013136214,123 Organic Reposado Tequila 750ml,Customer Square 2,0


In [25]:
list_of_retail_stores = ['Customer Square ' + str(x) for x in range(1,4)] #we have 3 stores active, we won't count the Warehouse
retail_names = ['Airport','Pleasant Valley','Oak Hill'] # make sure this matches the above list length - no known automation possible
store_map = dict(zip(list_of_retail_stores,retail_names))

my_pars['Location'] = my_pars.apply(lambda x: store_map[x.Location],axis=1 )
my_pars.head()

,SKU,ProductName,Location,Par
0,7503013136207,123 Organic Blanco Tequila 750ml,Airport,0
1,7503013136207,123 Organic Blanco Tequila 750ml,Pleasant Valley,0
2,7503013136207,123 Organic Blanco Tequila 750ml,Oak Hill,0
3,7503013136214,123 Organic Reposado Tequila 750ml,Airport,0
4,7503013136214,123 Organic Reposado Tequila 750ml,Pleasant Valley,0


In [26]:
my = df.merge(right=my_pars,on=['SKU','ProductName','Location'])
#my = my.drop('Customer')
my.head()

,Category,SKU,ProductName,Location,OnHand,Par
0,Tequila,7503013136207,123 Organic Blanco Tequila 750ml,Airport,2.0,0
1,Tequila,7503013136207,123 Organic Blanco Tequila 750ml,Oak Hill,1.0,0
2,Tequila,7503013136207,123 Organic Blanco Tequila 750ml,Pleasant Valley,3.0,0
3,Tequila,7503013136214,123 Organic Reposado Tequila 750ml,Airport,0.0,0
4,Tequila,7503013136214,123 Organic Reposado Tequila 750ml,Oak Hill,1.0,0


In [27]:
host_store = 'Oak Hill'

my['Shy'] = my.Par - my.OnHand
my['Send'] = np.nan
for i, row in my.iterrows():
    temp = my[(my.Location == host_store) & (my.ProductName == row.ProductName)]
    if len(temp) > 0 and row.Shy > 0 and temp.iloc[0].Shy < 0:
        if row.Shy <= (temp.iloc[0].Shy*-1):
            my.at[i,'Send'] = row.Shy
        else:
            my.at[i,'Send'] = temp.iloc[0].Shy*-1

In [28]:
my[my.Send.isnull()==False].head(40)
my.to_csv('InventoryShuffleMINE.csv')

In [36]:
bloat = my[(my.Location==host_store) & (my.ProductName.str.contains(' 50ml')==False)].sort_values(by='Shy',ascending=True)
bloat.to_csv('bloatOH.csv')